In [1]:
%load_ext autoreload
%autoreload 2 

In [3]:
import json
import jsonpath
import traitlets
import ipywidgets
import ipytree
import logging
import airavata_cerebrum.util.desc_config as cbm_dcfg
import airavata_cerebrum.model.desc as cbm_desc
import airavata_cerebrum.view.tree as cbm_tree
import mousev1.model as v1model


[atl1-1-01-011-8-2.pace.gatech.edu:mpi_rank_0][MPID_Init] [Performance Suggestion]: Application has requested for multi-thread capability. If allocating memory from different pthreads/OpenMP threads, please consider setting MV2_USE_ALIGNED_ALLOC=1 for improved performance.
Use MV2_USE_THREAD_WARNING=0 to suppress this error message.



              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.8.0
 Built: Aug 27 2024 04:32:09

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [4]:
with open("./v1l4/description/db_connect_output.json") as dbf:
    db_out_data = json.load(dbf)

In [9]:
db_out_data['airavata_cerebrum.dataset.ai_synphys'][0].keys()

dict_keys(["('L4-Pyr', 'L4-Pyr')", "('L4-Pyr', 'L4-Pvalb')", "('L4-Pyr', 'L4-Sst')", "('L4-Pyr', 'L4-Vip')", "('L4-Pvalb', 'L4-Pyr')", "('L4-Pvalb', 'L4-Pvalb')", "('L4-Pvalb', 'L4-Sst')", "('L4-Pvalb', 'L4-Vip')", "('L4-Sst', 'L4-Pyr')", "('L4-Sst', 'L4-Pvalb')", "('L4-Sst', 'L4-Sst')", "('L4-Sst', 'L4-Vip')", "('L4-Vip', 'L4-Pyr')", "('L4-Vip', 'L4-Pvalb')", "('L4-Vip', 'L4-Sst')", "('L4-Vip', 'L4-Vip')"])

In [20]:
import ast
import numpy as np
import pandas as pd
#
with open("./v1l4/description/db_connect_output.json") as dbf:
    db_out_data = json.load(dbf)
ai_syn_data = db_out_data['airavata_cerebrum.dataset.ai_synphys'][0]
row_keys = set([ast.literal_eval(xk)[0] for xk in ai_syn_data.keys()])
col_keys = set([ast.literal_eval(xk)[1] for xk in ai_syn_data.keys()])
col_names = row_names = list(row_keys)
connect_matrix = np.zeros((len(row_names), len(col_names)))
for i, cx in enumerate(col_names):
    for j,rx in enumerate(row_names):
        connect_matrix[i, j] = ai_syn_data[repr((cx, rx))]
pd.DataFrame(connect_matrix, index=row_names, columns=col_names)


,L4-Pvalb,L4-Vip,L4-Pyr,L4-Sst
L4-Pvalb,0.610531,0.101648,0.356604,0.065354
L4-Vip,0.025412,0.034990,0.000000,0.188560
L4-Pyr,0.198113,0.049624,0.125985,0.045855
L4-Sst,0.392124,0.306504,0.287637,0.016196


In [ ]:
#
#
md_config = cbm_dcfg.ModelDescConfig(
    name="v1l4",
    base_dir="./",
    config_files={"config": ["config.json"]},
    config_dir="./v1l4/description/",
)
print("Checking if mandatory config keys exists : ", md_config.valid())

mdtp_config = cbm_dcfg.ModelDescConfigTemplate(
    config_files={"templates": ["config_template.json"]},
    config_dir="./v1l4/description/",
)
print("Config loaded with ", len(mdtp_config.get_templates()), " templates")

model_dex =  cbm_desc.ModelDescription(
    config=md_config,
    region_mapper=mousev1.V1RegionMapper,
    neuron_mapper=mousev1.V1NeuronMapper,
    connection_mapper=mousev1.V1ConnectionMapper,
    network_builder=mousev1.V1BMTKNetworkBuilder, 
    custom_mod="./v1l4/description/custom_mod.json", 
    save_flag=False,
)  

In [ ]:
## S

In [29]:
src_tree = cbm_tree.SourceDataTreeView(md_config, mdtp_config)
src_tree.build()
src_tree.layout

TwoByTwoLayout(children=(Tree(layout=Layout(grid_area='top-left', width='40%'), multiple_selection=False, node…

In [30]:
loc_tree = cbm_tree.D2MLocationsTreeView(md_config, mdtp_config)
loc_tree.build()
loc_tree.layout

TwoByTwoLayout(children=(Tree(layout=Layout(grid_area='top-left', width='40%'), multiple_selection=False, node…

In [32]:
conn_tree = cbm_tree.D2MConnectionsTreeView(md_config, mdtp_config)
conn_tree.build()
conn_tree.layout

TwoByTwoLayout(children=(Tree(layout=Layout(grid_area='top-left', width='40%'), multiple_selection=False, node…

In [4]:
import v1l4_view
#
dsrc_views = {}
for qx, dctx in mdtp_config.get_templates().items():
    # print("Initializing Panels for ", qx)
    dsrc_views[qx] = v1l4_view.QXTemplateView(dctx)
dbxc = md_config.get_config("source_data")
tree = v1l4_view.source_data_tree(dbxc)
tree.layout.width = '40%' 
full_panel = ipywidgets.TwoByTwoLayout(top_left=tree, bottom_right=None)

full_panel


/home/srirampc/anaconda3/envs/cerebrum/lib/python3.10/site-packages/neuroanalysis/util/jit.py:25: UserWarning: Could not import numba; falling back to slower implementation.
  warnings.warn("Could not import numba; falling back to slower implementation.")


TwoByTwoLayout(children=(Tree(layout=Layout(grid_area='top-left', width='40%'), multiple_selection=False, node…

In [5]:
#
def tree_update(change):
    new_val = change["new"]
    if not len(new_val):
        return
    logging.getLogger("tree-update").warning("Key : " + new_val[0].node_key)
    node_key = new_val[0].node_key
    if node_key in dsrc_views:
        logging.getLogger("tree-update").warning("Key in Panel ")
        rview = dsrc_views[node_key]
        rview.update(new_val[0])
        full_panel.bottom_right = rview.panel
tree.observe(tree_update, names='selected_nodes')

In [116]:
model_dex = v1l4.v1l4_model_desc(config_files={"config": "config.json", "templates": "config_template.json"})

INFO:airavata_cerebrum.atlas.log.logging:Processing xformers for db airavata_cerebrum.atlas.data.abm_celltypes
INFO:airavata_cerebrum.atlas.log.logging:Running XFormer : airavata_cerebrum.atlas.data.json_filter.IterJPatchFilter
INFO:airavata_cerebrum.atlas.log.logging:Completed XForm : airavata_cerebrum.atlas.data.json_filter.IterJPatchFilter
INFO:airavata_cerebrum.atlas.log.logging:Running XFormer : airavata_cerebrum.atlas.data.json_filter.IterJPatchFilter
INFO:airavata_cerebrum.atlas.log.logging:Completed XForm : airavata_cerebrum.atlas.data.json_filter.IterJPatchFilter
INFO:airavata_cerebrum.atlas.log.logging:Running XFormer : airavata_cerebrum.atlas.data.json_filter.IterJPointerFilter
INFO:airavata_cerebrum.atlas.log.logging:Completed XForm : airavata_cerebrum.atlas.data.json_filter.IterJPointerFilter
INFO:airavata_cerebrum.atlas.log.logging:Completed processing xformers for db airavata_cerebrum.atlas.data.abm_celltypes
INFO:airavata_cerebrum.atlas.log.logging:Processing xformers f

In [117]:
#
with open("./v1l4/description/db_connect_output.json") as ifx:
    jsx = json.load(ifx)
    for x, y in jsx.items():
        print(x, len(y))
#
with open("./v1l4/description/db_xform_output.json") as ifx:
    jsx = json.load(ifx)
    for x, y in jsx.items():
        print(x, len(y))
#
for x, y in xdb_output.items():
    print(x, len(y))
#
for x, y in xdb_output2.items():
    print(x, len(y))

airavata_cerebrum.atlas.data.abm_celltypes 457
airavata_cerebrum.atlas.data.abc_mouse 1
airavata_cerebrum.atlas.data.abm_celltypes 132
airavata_cerebrum.atlas.data.abc_mouse 1
airavata_cerebrum.atlas.data.abm_celltypes 132
airavata_cerebrum.atlas.data.abc_mouse 1
airavata_cerebrum.atlas.data.abm_celltypes 132
airavata_cerebrum.atlas.data.abc_mouse 1


In [118]:
import v1l4_view
cfg_template =  v1l4_view.get_model_template()
cfg_temp_views = {}
for qxkey, dctx in cfg_template.get_template().items():
    print(qxkey)
    cfg_temp_views[qxkey] = v1l4_view.QXTemplateView(dctx)

airavata_cerebrum.atlas.data.abm_celltypes.CTDbCellApiQuery
airavata_cerebrum.atlas.data.abm_celltypes.CTDbGlifApiQuery
airavata_cerebrum.atlas.data.abc_mouse.ABCDbMERFISH_CCFQuery
airavata_cerebrum.atlas.data.ai_synphys.AISynPhysQuery
airavata_cerebrum.atlas.data.base.TQDMWrapper
airavata_cerebrum.atlas.data.abm_celltypes.CTExplainedRatioFilter
airavata_cerebrum.atlas.data.abm_celltypes.CTPropertyFilter
airavata_cerebrum.atlas.data.abm_celltypes.CTModelNameFilter
airavata_cerebrum.atlas.data.abc_mouse.ABCDbMERFISH_CCFLayerRegionFilter
airavata_cerebrum.atlas.data.abc_mouse.ABCDbMERFISH_CCFFractionFilter


In [71]:
cfg_temp_views["airavata_cerebrum.atlas.data.abm_celltypes.CTDbCellApiQuery"].panel

In [72]:
cfg_temp_views["airavata_cerebrum.atlas.data.abm_celltypes.CTDbGlifApiQuery"].panel

In [23]:
qry_cfg_temp_panel["airavata_cerebrum.atlas.data.abc_mouse.ABCDbMERFISH_CCFQuery"].panel

In [73]:
cfg_temp_views["airavata_cerebrum.atlas.data.ai_synphys.AISynPhysQuery"].panel

In [74]:

cfg_temp_views["airavata_cerebrum.atlas.data.base.TQDMWrapper"].panel

In [75]:
cfg_temp_views["airavata_cerebrum.atlas.data.abm_celltypes.CTExplainedRatioFilter"].panel

In [119]:
cfg_temp_views["airavata_cerebrum.atlas.data.abm_celltypes.CTPropertyFilter"].panel

In [77]:
cfg_temp_views["airavata_cerebrum.atlas.data.abm_celltypes.CTModelNameFilter"].panel

In [78]:
cfg_temp_views["airavata_cerebrum.atlas.data.abc_mouse.ABCDbMERFISH_CCFLayerRegionFilter"].panel

In [32]:
xform_cfg_temp_panel["airavata_cerebrum.atlas.data.abc_mouse.ABCDbMERFISH_CCFFractionFilter"].panel

In [120]:
import v1l4_view
#
dbxc = model_dex.model_config.get_config("db_connect")
tree = v1l4_view.db_conn_tree(dbxc)
tree.layout.width = '40%' 
full_panel = ipywidgets.TwoByTwoLayout(top_left=tree, bottom_right=None)
#
def tree_update(change):
    new_val = change["new"]
    if not len(new_val):
        return
    logging.getLogger("tree-update").warning("Key : " + new_val[0].node_key)
    node_key = new_val[0].node_key
    if node_key in cfg_temp_views:
        logging.getLogger("tree-update").warning("Key in Panel ")
        rview = cfg_temp_views[node_key]
        full_panel.bottom_right = rview.panel
tree.observe(tree_update, names='selected_nodes')
full_panel

TwoByTwoLayout(children=(Tree(layout=Layout(grid_area='top-left', width='40%'), multiple_selection=False, node…

In [12]:
md_struct = model_dex.build_net_struct()
md_struct = model_dex.apply_custom_mod()
ntview = cbm_tree.NetworkTreeView(md_struct, left_width="55%")
ntx = ntview.build()

In [13]:
ntx.tree.observe(ntx.tree_update, names="selected_nodes")
ntx.layout

TwoByTwoLayout(children=(Tree(layout=Layout(grid_area='top-left', width='55%'), multiple_selection=False, node…

In [14]:
ntx.panel_dict.keys()

dict_keys(['v1l4', 'VISp4', "('4', 'Scnn1a')", '501023852', '323838579', '500964358', '395785266', '476266853', '320207387', '517647182', '325464516', '476218657', "('4', 'Rorb')", '322723785', '478586425', '329557305', '500859045', '479179020', '324065524', '322761772', '467357834', '329550277', '476562817', '466378900', '486472847', '480648382', '490626827', '479508018', '490626718', '484770879', '500894271', '485836906', '480124551', '469763389', '473020156', '486500736', '484679812', '467003163', "('4', 'Vip')", '501570114', '560690291', '562540395', '535728342', '531121585', '561934585', "('4', 'Pvalb')", '487405644', '478793814', '475894121', '571314481', '478958894', '569997187', '579414994', '481127348', '491038812', '579672464', '488708322', '341459814', '481127173', '487667205', '572375809', "('4', 'Sst')", '476099282', '501282204', '485061364', '501280518', '573622646', '476753465', "(('4', 'Pvalb'), ('4', 'Pvalb'))", "(('4', 'Pvalb', ''), ('4', 'Pvalb', '478793814'))", "(('